<a href="https://colab.research.google.com/github/SimplySnap/PDLP-AMD-RIPS/blob/connorphillips700-patch-1/PDHG_torch_pulp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 18.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import numpy as np
from pulp import LpProblem, LpConstraintEQ, LpConstraintGE, LpConstraintLE, LpAffineExpression

In [10]:
from time import perf_counter

class Timer:
    def __enter__(self):
        self.start = perf_counter()
        return self
    def __exit__(self, *args):
        self.end = perf_counter()
        self.elapsed = self.end - self.start
        print(f"Elapsed time: {self.elapsed:.6f} seconds")

In [5]:
def project_lambda_box(grad, is_neg_inf, is_pos_inf):
    """
    Projects the gradient onto the normal cone of the feasible region defined by bounds l and u.

    For each i:
      - If l[i] == -inf and u[i] == +inf: projection is 0
      - If l[i] == -inf and u[i] is real: clamp to ≤ 0 (R⁻)
      - If l[i] is real and u[i] == +inf: clamp to ≥ 0 (R⁺)
      - If both are finite: no projection (keep full value)

    Args:
        grad: (n, 1) gradient vector (torch tensor)
        l: (n, 1) lower bounds (torch tensor)
        u: (n, 1) upper bounds (torch tensor)

    Returns:
        projected: (n, 1) projected gradient (interpreted as λ)
    """
    projected = torch.zeros_like(grad)

    # Case 1: (-inf, +inf) → {0}
    unconstrained = is_neg_inf & is_pos_inf
    projected[unconstrained] = 0.0

    # Case 2: (-inf, real) → R⁻ → clamp at 0 from above
    neg_only = is_neg_inf & ~is_pos_inf
    projected[neg_only] = torch.clamp(grad[neg_only], max=0.0)

    # Case 3: (real, +inf) → R⁺ → clamp at 0 from below
    pos_only = ~is_neg_inf & is_pos_inf
    projected[pos_only] = torch.clamp(grad[pos_only], min=0.0)

    # Case 4: (real, real) → full space → keep gradient
    fully_bounded = ~is_neg_inf & ~is_pos_inf
    projected[fully_bounded] = grad[fully_bounded]

    return projected


In [7]:
def pdhg_torch(c, G, h, A, b, l, u, is_neg_inf, is_pos_inf, l_dual, u_dual, device, max_iter=1000, tol=1e-4, verbose=True):
    """
    Solves:
        min cᵀx s.t. Gx ≥ h, Ax = b, l ≤ x ≤ u
    using the Primal-Dual Hybrid Gradient (PDHG) algorithm.
    """
    n = c.shape[0]
    m_ineq = G.shape[0] if G.numel() > 0 else 0
    m_eq = A.shape[0] if A.numel() > 0 else 0

    # Combine constraints
    combined_matrix_list = []
    rhs = []
    if m_ineq > 0:
        combined_matrix_list.append(G)
        rhs.append(h)
    if m_eq > 0:
        combined_matrix_list.append(A)
        rhs.append(b)

    if not combined_matrix_list:
        raise ValueError("Both G and A matrices are empty.")

    K = torch.vstack(combined_matrix_list).to(device)           # Combined constraint matrix
    q = torch.vstack(rhs).to(device)                            # Combined right-hand side
    c = c.to(device)

    q_norm = torch.linalg.norm(q, 2)
    c_norm = torch.linalg.norm(c, 2)


    eta = 0.9 / torch.linalg.norm(K, 2)
    omega = 1.0

    tau = eta / omega
    sigma = eta * omega

    theta = 1.0

    # Initialize primal and dual
    x = torch.zeros((n, 1), device=device)
    x_old = x.clone()
    y = torch.zeros((K.shape[0], 1), device=device)

    for k in range(max_iter):
        x_old.copy_(x)

        # Compute gradient and primal update
        Kt_y = K.T @ y
        grad = c - Kt_y
        x = torch.clamp(x - tau * grad, min=l, max=u)

        # Extrapolate
        x_bar = x + theta * (x - x_old)

        # Dual update
        K_xbar = K @ x_bar
        y += sigma * (q - K_xbar)

        # Project duals:
        if m_ineq > 0:
            y[:m_ineq] = torch.clamp(y[:m_ineq], min=0.0)

        # --- Check Termination Every 10 Iterations ---
        if k % 10 == 0:
            # Primal and dual objective
            prim_obj = (c.T @ x)[0][0]
            dual_obj = (q.T @ y)[0][0]

            # Lagrange multipliers from box projection
            lam = project_lambda_box(grad, is_neg_inf, is_pos_inf)
            lam_pos = (l_dual.T @ torch.clamp(lam, min=0.0))[0][0]
            lam_neg = (u_dual.T @ torch.clamp(lam, max=0.0))[0][0]

            adjusted_dual = dual_obj + lam_pos + lam_neg
            duality_gap = abs(adjusted_dual - prim_obj)

            # Primal residual (feasibility)
            residual_eq = A @ x - b if m_eq > 0 else torch.zeros(1, device=device)
            residual_ineq = torch.clamp(h - G @ x, min=0.0) if m_ineq > 0 else torch.zeros(1, device=device)
            primal_residual = torch.norm(torch.vstack([residual_eq, residual_ineq]), p=2).item()

            # Dual residual (change in x)
            dual_residual = torch.norm(grad - lam, p=2).item()

            if verbose:
                print(f"[{k}] Primal Obj: {prim_obj:.4f}, Adjusted Dual Obj: {adjusted_dual:.4f}, "
                      f"Gap: {duality_gap:.2e}, Prim Res: {primal_residual:.2e}, Dual Res: {dual_residual:.2e}")

            # Termination condition
            if (primal_residual <= tol * (1 + q_norm) and
                dual_residual <= tol * (1 + c_norm) and
                duality_gap <= tol * (1 + abs(prim_obj) + abs(adjusted_dual))):
                if verbose:
                    print(f"Converged at iteration {k}")
                break

    return x


In [18]:
def mps_to_standard_form_torch(mps_file, device='cpu'):
    """
    Reads an MPS file and converts the linear programming problem into standard form tensors.
    The standard form is:
    minimize     c^T * x
    subject to   G * x >= h
                 A * x = b
                 l <= x <= u
    """
    # fromMPS returns the problem and a dictionary mapping variable names to LpVariable objects
    _, prob = LpProblem.fromMPS(mps_file)
    var_list = prob.variables()
    num_vars = len(var_list)
    name_to_idx = {var.name: i for i, var in enumerate(var_list)}

    # --- Bounds (l, u) ---
    l = [-np.inf if v.lowBound is None else v.lowBound for v in var_list]
    u = [np.inf if v.upBound is None else v.upBound for v in var_list]

    # --- Objective (c) ---
    c = np.array([prob.objective.get(v, 0.0) for v in var_list])

    # --- Constraints (A, b, G, h) ---
    A_rows, G_rows = [], []
    b_eq, b_ineq = [], []

    for con in prob.constraints.values():
        row = np.zeros(num_vars)
        for var, coeff in con.items():
          idx = name_to_idx[var.name]
          row[idx] = coeff

        # PuLP stores the RHS constant on the left side, so we negate it
        rhs = -con.constant

        if con.sense == LpConstraintEQ:
            # Ax = b
            A_rows.append(row)
            b_eq.append(rhs)
        elif con.sense == LpConstraintGE:
            # Gx >= h
            G_rows.append(row)
            b_ineq.append(rhs)
        elif con.sense == LpConstraintLE:
            # Gx <= h
            G_rows.append(-row)
            b_ineq.append(-rhs)

    # --- Convert to PyTorch Tensors ---
    c_tensor = torch.tensor(c, dtype=torch.float32, device=device).view(-1, 1)
    l_tensor = torch.tensor(l, dtype=torch.float32, device=device).view(-1, 1)
    u_tensor = torch.tensor(u, dtype=torch.float32, device=device).view(-1, 1)

    A_tensor = torch.tensor(A_rows, dtype=torch.float32, device=device) if A_rows else torch.zeros((0, num_vars), device=device)
    b_tensor = torch.tensor(b_eq, dtype=torch.float32, device=device).view(-1, 1) if b_eq else torch.zeros((0, 1), device=device)

    G_tensor = torch.tensor(G_rows, dtype=torch.float32, device=device) if G_rows else torch.zeros((0, num_vars), device=device)
    h_tensor = torch.tensor(b_ineq, dtype=torch.float32, device=device).view(-1, 1) if b_ineq else torch.zeros((0, 1), device=device)

    return c_tensor, G_tensor, h_tensor, A_tensor, b_tensor, l_tensor, u_tensor

In [24]:
if __name__ == '__main__':
    # --- Configuration ---
    mps_file_path = 'ActualSample.mps'

    # --- Device Selection ---
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"PyTorch is using ROCm/CUDA device: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device('cpu')
        print("ROCm/CUDA not available. PyTorch is using CPU.")

    # --- Data Loading ---
    try:
        c, G, h, A, b, l, u = mps_to_standard_form_torch(mps_file_path, device=device)
    except Exception as e:
        print(f"Failed to load MPS file: {e}")
        exit(1)
        
    with Timer():

        is_neg_inf = torch.isinf(l) & (l < 0)
        is_pos_inf = torch.isinf(u) & (u > 0)

        l_dual = l.clone()
        u_dual = u.clone()

        l_dual[is_neg_inf] = 0
        u_dual[is_pos_inf] = 0

        # --- Run PDHG Solver on the GPU or CPU ---
        solution = pdhg_torch(c, G, h, A, b, l, u, is_neg_inf, is_pos_inf, l_dual, u_dual, device=device)

        print("\nSolution (first 10 variables):")
        print(solution[:10].cpu().numpy())

ROCm/CUDA not available. PyTorch is using CPU.
[0] Primal Obj: -1.7264, Adjusted Dual Obj: 1821.6095, Gap: 1.82e+03, Prim Res: 9.40e+02, Dual Res: 2.90e+01
[10] Primal Obj: -18.1179, Adjusted Dual Obj: 20028.1484, Gap: 2.00e+04, Prim Res: 9.39e+02, Dual Res: 5.01e+01
[20] Primal Obj: -31.2154, Adjusted Dual Obj: 38190.0156, Gap: 3.82e+04, Prim Res: 9.37e+02, Dual Res: 8.75e+01
[30] Primal Obj: -39.9071, Adjusted Dual Obj: 56269.6602, Gap: 5.63e+04, Prim Res: 9.32e+02, Dual Res: 1.27e+02
[40] Primal Obj: -44.7730, Adjusted Dual Obj: 74231.0312, Gap: 7.43e+04, Prim Res: 9.26e+02, Dual Res: 1.66e+02
[50] Primal Obj: -46.8275, Adjusted Dual Obj: 92039.8438, Gap: 9.21e+04, Prim Res: 9.18e+02, Dual Res: 2.04e+02
[60] Primal Obj: -47.2230, Adjusted Dual Obj: 109664.2031, Gap: 1.10e+05, Prim Res: 9.09e+02, Dual Res: 2.41e+02
[70] Primal Obj: -46.8787, Adjusted Dual Obj: 127074.1719, Gap: 1.27e+05, Prim Res: 8.98e+02, Dual Res: 2.76e+02
[80] Primal Obj: -46.0398, Adjusted Dual Obj: 144240.5781,